In [1]:
import cv2
import time
import mediapipe
import numpy as np

In [2]:
result=cv2.VideoWriter('poselandmarks.avi',cv2.VideoWriter_fourcc(*'XVID'),30,(640,480))

In [3]:
mp_pose=mediapipe.solutions.pose
mpDraw=mediapipe.solutions.drawing_utils

In [4]:
def calc_angle(x,y,z):
    x=np.array(x)
    y=np.array(y)
    z=np.array(z)

    radians=np.arctan2(z[1]-y[1],z[0]-y[0])-np.arctan2(x[1]-y[1],x[0]-y[0])
    angle=np.abs(radians*180/np.pi)
    if angle>180:
        angle=360-angle

    return angle  

In [5]:
for lm in mp_pose.PoseLandmark:
    print(list(mp_pose.PoseLandmark).index(lm),lm,sep='\t')

0	0
1	1
2	2
3	3
4	4
5	5
6	6
7	7
8	8
9	9
10	10
11	11
12	12
13	13
14	14
15	15
16	16
17	17
18	18
19	19
20	20
21	21
22	22
23	23
24	24
25	25
26	26
27	27
28	28
29	29
30	30
31	31
32	32


In [15]:
cap=cv2.VideoCapture('cable overhead curl.mp4')
stage=None
prev_angle1=0
prev_angle2=0
correct=0
incorrect=0
done=False
notdone=False
elbow_done=False
elbow_notdone=False
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        #Read the video and its dimensions
        ret,frame=cap.read()
        frame_height, frame_width, _ = frame.shape

        
        #convert the frame of video to RGB
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
        #do the process in the converted frame to extract the landmarks
        results=pose.process(image)
        
        #convert the frame to BGR to show
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        try:
            #extract lanmarks
            landmarks=results.pose_landmarks.landmark
            
            #coordinates of wrists , elbows , shoulders and hips 
            Left_shoulder=[landmarks[11].x,landmarks[11].y]
            Right_shoulder=[landmarks[12].x,landmarks[12].y]
            Left_elbow=[landmarks[13].x,landmarks[13].y]
            Right_elbow=[landmarks[14].x,landmarks[14].y] 
            Left_wrist=[landmarks[15].x,landmarks[15].y]
            Right_wrist=[landmarks[16].x,landmarks[16].y]
            Left_hip=[landmarks[23].x,landmarks[23].y]
            Right_hip=[landmarks[24].x,landmarks[24].y]
            
            #calculate the angles 
            Left_shoulder_angle=calc_angle(Left_hip,Left_shoulder,Left_elbow)
            Right_shoulder_angle=calc_angle(Right_hip,Right_shoulder,Right_elbow)
            Left_elbow_angle=calc_angle(Left_shoulder,Left_elbow,Left_wrist)
            Right_elbow_angle=calc_angle(Right_shoulder,Right_elbow,Right_wrist)

            #show the angles in the frame
            cv2.putText(image,str(round(Left_shoulder_angle,2)),tuple(np.multiply(Left_shoulder,[frame_width,frame_height]).astype(int)),cv2.FONT_HERSHEY_PLAIN,1.5,(255,255,255)) 
            cv2.putText(image,str(round(Right_shoulder_angle,2)),tuple(np.multiply(Right_shoulder,[frame_width,frame_height]).astype(int)),cv2.FONT_HERSHEY_PLAIN,1.5,(255,255,255)) 
            cv2.putText(image,str(round(Left_elbow_angle,2)),tuple(np.multiply(Left_elbow,[frame_width,frame_height]).astype(int)),cv2.FONT_HERSHEY_PLAIN,1.5,(255,255,255))
            cv2.putText(image,str(round(Right_elbow_angle,2)),tuple(np.multiply(Right_elbow,[frame_width,frame_height]).astype(int)),cv2.FONT_HERSHEY_PLAIN,1.5,(255,255,255))  
            
            #two sides
            if round(Left_elbow_angle)>=prev_angle1:
                prev_angle1=round(Left_elbow_angle)
                continue
            else:
                if Left_elbow_angle>90 and Right_elbow_angle>90 and 65<=Left_shoulder_angle<=135 and 65<=Right_shoulder_angle<=135 :
                    if stage =='UP'or stage==None :
                        stage='DOWN' 
                        if done==True and notdone==False:
                            correct+=1
                            done=False
                        if notdone==True and done==False :
                            incorrect+=1
                            notdone=False    
                        prev_angle1=round(Left_elbow_angle)
                        prev_angle2=round(Left_elbow_angle)
            if round(Left_elbow_angle)<=prev_angle2:
                prev_angle2=round(Left_elbow_angle)
                continue
            else:

                if (Left_elbow_angle<=75 and 65<=Left_shoulder_angle<=135) and (Right_elbow_angle<=75 and 65<=Right_shoulder_angle<=135 ) and stage=='DOWN' and done ==False:
                    stage='UP'
                    done=True

                elif ((90>=Left_elbow_angle>75 or Left_shoulder_angle<65 or Left_shoulder_angle>135 )or(90>=Right_elbow_angle>75 or Right_shoulder_angle<65 or Right_shoulder_angle>135)) and stage=='DOWN' and notdone==False :
                    if prev_angle2<=75:
                            elbow_done=True    
                            stage='UP'
                    if 90>=Left_elbow_angle>75 or 90>=Right_elbow_angle>75:
                        cv2.rectangle(image,(0,70),(300,100),(0,255,255),-1)
                        cv2.putText(image,"narrow your elbows",(0,95),cv2.FONT_HERSHEY_PLAIN,1.5,(0,0,0),cv2.LINE_4)
                        stage='UP'
                        notdone=True
                        elbow_notdone=True

                    if Left_shoulder_angle<65 or Left_shoulder_angle>135 or Right_shoulder_angle<65 or Right_shoulder_angle>135:
                        cv2.rectangle(image,(0,110),(400,140),(150,150,230),-1)
                        if Left_shoulder_angle<65 or Right_shoulder_angle<65:
                            cv2.putText(image,"keep your arms wide",(0,135),cv2.FONT_HERSHEY_PLAIN,1.5,(0,0,0),cv2.LINE_4)
                        elif Left_shoulder_angle>135 or Right_shoulder_angle>135:
                            cv2.putText(image,"keep your arms narrow",(0,135),cv2.FONT_HERSHEY_PLAIN,1.5,(0,0,0),cv2.LINE_4)    
                        if elbow_done==True or elbow_notdone==True :
                            notdone=True   

                    elbow_done=False 
                    elbow_notdone=False 
                prev_angle1=round(Left_elbow_angle)
                prev_angle2=round(Left_elbow_angle)
           

            #create the main rectangles
            cv2.rectangle(image, (0,0), (200,50), (245,117,16), -1)
            cv2.rectangle(image, (frame_width-150,30), (frame_width,80), (0,255,0), -1)
            cv2.rectangle(image, (frame_width-150,120), (frame_width,170), (0,0,255), -1)        
            
            # Rep data 
            cv2.putText(image, ' Correct:', (frame_width-145,55),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(correct), (frame_width-40,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            # Rep data incorrect
            cv2.putText(image, ' InCorrect:', (frame_width-150,145), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(incorrect), (frame_width-30,148), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE:', (10,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA) # img, name, position, font
            cv2.putText(image, stage, (120,25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)      
        
        except:
            pass

        #draw the landmarks and draw the connections between them.    
        mpDraw.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                              mpDraw.DrawingSpec(color=(240,100,100),thickness=4),
                              mpDraw.DrawingSpec(color=(50,50,240),thickness=2))
        
        #show the frame     
        
        result.write(image)
        cv2.imshow("frame",image)
        
        #to close
        key=cv2.waitKey(1)
        if key==27:
            break
    cv2.destroyAllWindows()
    cap.release()           